# Mise en forme des données dans un dataframe

Notre premiere idée consistait a récuperer les données dans un dataframe. Cette méthode a bien évidemment vite mopntré ses limites, car récuperer toutes les données prenait plusiseurs heures, et il était donc essentiel de les enregistrer dans un CSV.

## Création du dataframe

In [2]:
import pandas as pd

In [3]:
dfQuestions = pd.DataFrame(
        {
            "id"  : [],
            "Groupe Parlementaire" : [],
            "Nom du député" : [],
            "Question" : []
        })

In [4]:
cols =  ["id",
            "Groupe Parlementaire",
            "Nom du député",
            "Question"]

#### Imports et fonctions utilitaires

In [5]:
!pip install webdriver-manager
!pip install -q lxml
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=9aca3753a7016bde476c9e453ec16494cfafe43141f26bb8dac3d4dd50873606
  Stored in directory: /Users/iokanaanbelfis-simon/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [2]:
import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

## Récupération des données d'une question

Nous avons défini les éléments suivants, essentiels à récupérer :
- l'identifiant de la question, `id` : afin d'avoir une clé primaire
- le texte de la question, `question` : pour des raisons évidentes
- le parti politique dont le député qui a posé la question est ressortissant `groupe` : pour avoir de quoi définir l'opinion politique de la question

Au delà de ces trois éléments là, il nous est également paru important de récolter :
- la rubrique `rubrique` et le titre `titre` de la question : qui peuvent ajouter des éléments importants à prendre en compte en plus du texte de la question
- le nom du député `nom` : Il est important pour deux raisons, car il permet de faire un lien avec la base de données des députés, et il est essentiel de l'enlever du texte de la question (pour ne pas que le programme ne s'intéresse qu'à cette partie là du texte pour determiner de quel parti il est issu)
- le ministere intérrogé `ministere` : pour les mêmes raisons que `rubrique` et `titre`
- la date de la quetsion `date` : qui pourrait çetre intéressante pour étudier l'évolution des thèmes du discours politique des différents partis


#### Ouverture de la Page

In [3]:
def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"

In [4]:
k = 1
source_code = request.urlopen(pageurl(k)).read()
page = bs4.BeautifulSoup(source_code)

#### Récupération d'une question

In [6]:
question0 = page.find("div", {"class" : "question"})
question = question0.find("p")
question = question.text.strip()

question

"M. Christophe Naegelen interroge M. le ministre de l'agriculture et de la souveraineté alimentaire sur le statut particulier des « patous » communément appelés chiens de berger ainsi que sur les dispositifs mis en place pour permettre la cohabitation entre les randonneurs et les patous. Les chiens de berger protègent les troupeaux et ne sont pas considérés comme divaguant lorsqu'ils se trouvent sans laisse et à distance de leur maître depuis la loi n° 2005-157 du 23 février 2005 qui a modifié l'article L. 211-23 du code rural. Désormais, la loi libère « de toute poursuite pénale pour divagation, l'éleveur ou le propriétaire d'un chien affecté à la protection du troupeau (« patous ») ou à la garde de celui-ci (chien de conduite) ». Des études ont montré que les patous ne sont pas dangereux pour les hommes, ces chiens sont dressés pour dissuader et non pour blesser. Néanmoins, des cas d'agressions sont recensés et une forte part des randonneurs sont inquiets de croiser ces chiens. Des p

#### Récupération de la Rubrique et du Titre de la question

In [7]:
rubrique0 = page.findAll("div", {"class" : "question_col33"})
rubrique,titre = rubrique0[0].find("p"), rubrique0[1].find("p")
rubrique, titre = rubrique.text.strip()[11:], titre.text.strip()[8:]

rubrique, titre

('animaux', 'Cohabitation entre les randonneurs et les patous')

#### Récupération du ministère interrogé

In [8]:
ministere0 = page.find("div", {"class" : "question_col50"})
ministere = ministere0.text.strip()[23:]

ministere

'Agriculture et souveraineté alimentaire'

#### Récupération de la date de la question

In [9]:
date0 = page.find("div", {"class" : "question_publish_date"})
date = date0.find("span")
date = date.text.strip()

date

'05/07/2022'

#### Récuperation des données du parlementaire

Pour récupérer les données du parlementaire, il est compliqué de le faire depuis la page même de la question : en effet, le nom, le groupe et la circonscription sont au même endroit, et il est difficile de les isoler (prénoms composés, noms de groupe avec virgules ou tirets ...)

Une solution plus simple est de se rendre sur la page associée au parlementaire, et d'y récuperer les questions qui y sont ici bien séparées.

In [10]:
lien0 = page.find("span", {"class" : "question_big_content"})
lien = lien0.find("a").get("href")

In [11]:
source_code_parlementaire = request.urlopen(lien).read()
page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

In [13]:
groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
groupe = groupe0.text.strip()

nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
nom = nom0.text.strip()

groupe, nom

('Libertés, Indépendants, Outre-mer et Territoires', 'M. Christophe Naegelen')

#### Ajout de la donnée au dataframe

In [16]:
dfQuestion = pd.DataFrame([[k, groupe, nom, question]], columns = cols)

In [17]:
dfQuestions = pd.concat([dfQuestions, dfQuestion], ignore_index = True)
dfQuestions

,id,Groupe Parlementaire,Nom du député,Question
0,1.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...


## Création du Data Frame

In [18]:
import pandas as pd
import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"

In [19]:
n = 100

In [20]:
dfQuestions = pd.DataFrame(
        {
            "id"  : [],
            "Groupe Parlementaire" : [],
            "Nom du député" : [],
            "Question" : []
        })

for k in range(1, n+1) : 
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    dfQuestion = pd.DataFrame([[k, groupe, nom, question]], columns = cols)
    dfQuestions = pd.concat([dfQuestions, dfQuestion], ignore_index = True)

dfQuestions

,id,Groupe Parlementaire,Nom du député,Question
0,1.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...
1,2.0,Les Républicains,M. Pierre Cordier,M. Pierre Cordier appelle l'attention de Mme l...
2,3.0,Démocrate (MoDem et Indépendants),M. Frédéric Petit,M. Frédéric Petit appelle l'attention de M. le...
3,4.0,Renaissance,Mme Marie-Pierre Rixain,Mme Marie-Pierre Rixain appelle l'attention de...
4,5.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...
...,...,...,...,...
95,96.0,Les Républicains,Mme Véronique Louwagie,Mme Véronique Louwagie attire l'attention de M...
96,97.0,La France insoumise - Nouvelle Union Populaire...,M. François Ruffin,M. François Ruffin interroge M. le ministre dé...
97,98.0,Les Républicains,M. Thibault Bazin,M. Thibault Bazin appelle l'attention de M. le...
98,99.0,Renaissance,Mme Véronique Riotton,Mme Véronique Riotton attire l'attention de M....


## Création d'un fichier csv (semicolon)

In [18]:
données = open("data_TEST.csv", "w")

données.write("id|groupe|nom|ministère_interrogé|rubrique|titre|date|question")

données.close()

In [19]:
données = open("data_TEST.csv", "a")

for k in range(1, 51) : 
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    rubrique0 = page.findAll("div", {"class" : "question_col33"})
    rubrique,titre = rubrique0[0].find("p"), rubrique0[1].find("p")
    rubrique, titre = rubrique.text.strip()[11:], titre.text.strip()[8:]
    
    ministere0 = page.find("div", {"class" : "question_col50"})
    ministere = ministere0.text.strip()[23:]

    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    date0 = page.find("div", {"class" : "question_publish_date"})
    date = date0.find("span")
    date = date.text.strip()
    
    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    données.write("\n" + str(k) + "|" + groupe + "|" + nom +  "|" + ministere + "|" + rubrique + "|" + titre + "|" + date + "|" + question)

données.close()

On peut également remarquer un problème d'encodage, les accents causent des problemes d'affichage dans le fichier.